In [1]:
import pandas as pd
from openpyxl import load_workbook, Workbook
import xlsxwriter
from datetime import date
fecha_actual = date.today()

# Convertir a string con formato personalizado
fecha_como_string = fecha_actual.strftime('%d-%m-%Y') 

In [2]:
df=pd.read_excel('ReporteCovid-19 (4).xlsx',sheet_name="Reporte Covid",index_col=0,skiprows=4)
df.shape

(7302, 36)

Validaciones

In [3]:
validaciones = df.pivot_table(
    values=['Nro. de Dosis'],
    index=['Municipio','Establecimiento'],
    columns='Estado Validación',
    aggfunc={'Nro. de Dosis': 'count'}, # Renombre la clave a 'Estado_Validacion_Conteo'
    margins=True
    # margins_names=True,  # Incluir totales generales y subtotales
    # margins_names={'all': 'Total'}  # Personalizar nombres de totales
)
validaciones

Nro. de Dosis               
Estado Validación                    NO VALIDADO VALIDADO   All
Municipio Establecimiento                                      
BERMEJO   21 DE DICIEMBRE                    NaN     78.0    78
          ARROZALES                          NaN      3.0     3
          AZUCARERO                          NaN     45.0    45
          BARREDERO                          NaN      2.0     2
          CAJA PETROLERA BERMEJO             NaN     12.0    12
...                                          ...      ...   ...
YUNCHARA  PASAJES                            NaN      9.0     9
          SANTA BARBARA DE ÑOQUERA           NaN      6.0     6
          TOJO                               NaN      5.0     5
          YUNCHARA                           NaN     11.0    11
All                                         20.0   7282.0  7302

[164 rows x 3 columns]

In [4]:
valida=df[df['Estado Validación']=='NO VALIDADO']
valida=valida.groupby(['Municipio','Establecimiento', 'Nombre Vacuna', 'Nro. de Dosis','Nombre Completo'
                          ,'Edad (Años)','Fecha de Vacunación','Fecha de Nacimiento','Usuario','Estado Validación']).size()
valida

Municipio  Establecimiento  Nombre Vacuna  Nro. de Dosis  Nombre Completo               Edad (Años)  Fecha de Vacunación  Fecha de Nacimiento  Usuario      Estado Validación
BERMEJO    CENTRAL          COVID-19       DOSIS ANUAL    CARLA MARCELA -- BRAÑEZ       29           01/04/2025           15/05/1995           agudiño      NO VALIDADO          1
                                                          ERICK DEL CASTILLO MOSQUEIRA  40           07/04/2025           21/02/1985           mfernandezj  NO VALIDADO          1
                                                          JULIO CESAR MIRANDA GUZMAN    39           03/04/2025           21/12/1985           mfernandezj  NO VALIDADO          1
                                                          MAYERLIN KRAMER NUÑEZ         36           22/04/2025           02/10/1988           agudiño      NO VALIDADO          1
           MOTO MENDEZ      COVID-19       DOSIS ANUAL    LUZ ANTONETTE ANGULO CADENA   15           04/04/202

In [5]:
valida1=df[df['Estado Validación']=='NO VALIDADO']
valida1=valida1.groupby(['Municipio','Establecimiento']).size()
valida1

Municipio  Establecimiento
BERMEJO    CENTRAL            4
           MOTO MENDEZ        2
PADCAYA    SALADO CONCHAS     9
           SAN ROQUE          3
YACUIBA    SAN JERONIMO       2
dtype: int64

Duplicados

In [6]:
duplicados2 = df.duplicated(subset=['Nombre Completo','Nombre Vacuna', 'Nro. de Dosis'
                          ,'Edad (Años)','Fecha de Nacimiento'])

# Mostrar solo las filas duplicadas
filas_duplicadas = df[duplicados2]
filas_duplicadas=filas_duplicadas.groupby(['Municipio','Establecimiento','Nombre Completo','Nombre Vacuna', 'Nro. de Dosis'
                          ,'Edad (Años)','Fecha de Nacimiento','Fecha de Vacunación','Usuario','Enfemedad de Base']).size()
filas_duplicadas

Municipio       Establecimiento                  Nombre Completo                Nombre Vacuna  Nro. de Dosis  Edad (Años)  Fecha de Nacimiento  Fecha de Vacunación  Usuario        Enfemedad de Base
CARAPARI        CAMPO LARGO HITO 22              JOSE SANTOS MARTINEZ MAMANI    COVID-19       DOSIS ANUAL    35           18/11/1989           22/01/2025           aguzmanv       NO                   1
                                                 MARTIN FRANKLIN JURADO FLORES  COVID-19       DOSIS ANUAL    28           23/02/1996           27/01/2025           aguzmanv       NO                   1
                                                 NESTALY YELVI JURADO FLORES    COVID-19       DOSIS ANUAL    26           04/03/1998           27/01/2025           aguzmanv       NO                   1
                LAS SIDRAS                       EDUARDO MARTINEZ CRUZ          COVID-19       DOSIS ANUAL    37           24/09/1987           11/02/2025           squispech1967  NO           

Personal de Salud menores a 18 años

In [7]:
sisaal=df[df['Personal de Salud']=='SI']
sisaal=sisaal[sisaal['Edad (Años)']<18]
df_salu = sisaal.groupby(['Municipio','Establecimiento', 'Nombre Vacuna', 'Nro. de Dosis','Nombre Completo'
                          ,'Edad (Años)','Fecha de Nacimiento','Personal de Salud','Fecha de Vacunación','Usuario']).size()
df_salu

Series([], dtype: int64)

Registros antes del Acto de Vacunacion

In [8]:
# Convert the dates to DateTime format
df["Fecha de Vacunación"] = pd.to_datetime(df["Fecha de Vacunación"], format="%d/%m/%Y")
df["Fecha Registro"] = pd.to_datetime(df["Fecha Registro"])

# Calculate the difference in days
df["Diferencia en Días"] = df["Fecha Registro"] - df["Fecha de Vacunación"]
df["Diferencia en Días"] =df["Diferencia en Días"].dt.days
dif_dia=df[df["Diferencia en Días"]<0]
dif_dia=dif_dia.groupby(['Municipio','Establecimiento','Nombre Completo', 'Nombre Vacuna', 'Nro. de Dosis'
                          ,'Edad (Años)','Fecha de Nacimiento','Fecha de Vacunación','Fecha Registro','Diferencia en Días','Usuario']).size()
dif_dia

Series([], dtype: int64)

Sin jeringa

In [9]:
snj1=df[(df['Jeringa de Administración'].isnull())]
df_snj=snj1.groupby(['Municipio','Establecimiento','Nombre Completo','Fecha de Vacunación','Fecha de Nacimiento','Edad (Años)','Nombre Vacuna', 'Nro. de Dosis'
                       ,'Usuario',]).size()
df_snj

Series([], dtype: int64)

Con diluyente

In [10]:
Con_dil=df[~(df['Jeringa de Dilusión'].isnull())& ~(df['Lote Diluyente'].isnull())]
Con_dil=Con_dil.groupby(['Municipio','Establecimiento','Nombre Completo','Fecha de Vacunación','Fecha de Nacimiento','Edad (Años)','Nombre Vacuna', 'Nro. de Dosis','Lote Diluyente'
                       ,'Jeringa de Dilusión','Usuario']).size()
Con_dil

Series([], dtype: int64)

In [11]:
# Crear un archivo Excel con varias hojas
with pd.ExcelWriter('COVID-'+fecha_como_string+'.xlsx',engine='xlsxwriter') as writer:
    workbook = writer.book
    format1 = workbook.add_format({'bold': True, 'font_size': 16,'font_color': 'red', 'align': 'center'})
    ####### DUPLICADOS###############
    # duplicados.to_excel(writer, sheet_name='duplicados', index=True, startrow=1, startcol=1)
    # dupsheet = writer.sheets['duplicados']    
    # dupsheet.write('D1', 'REGISTROS DUPLICADOS', format1)
    ########################
    filas_duplicadas.to_excel(writer, sheet_name='duplicadoss', index=True, startrow=1, startcol=1)
    fdupsheet = writer.sheets['duplicadoss']    
    fdupsheet.write('D1', 'REGISTROS DUPLICADOSS', format1)
    #######
    sisaal.to_excel(writer, sheet_name='Per_Sald_<18', index=True, startrow=1, startcol=1)
    sndysheet = writer.sheets['Per_Sald_<18']    
    sndysheet.write('D1', 'PERSONAL DE SALUD MENOR A 18 AÑOS', format1)
    #######
    valida.to_excel(writer, sheet_name='validaciones',index=True, startrow=1, startcol=1)
    snjsheet = writer.sheets['validaciones']    
    snjsheet.write('D1', 'REGISTROS NO VALIDADOS', format1)
    #######
    validaciones.to_excel(writer, sheet_name='validaciones_num',index=True, startrow=1, startcol=1)
    vacsheet = writer.sheets['validaciones_num']    
    vacsheet.write('D1', 'REGISTROS NO VALIDADOS', format1)
    ########
    valida1.to_excel(writer, sheet_name='novali_es',index=True, startrow=1, startcol=1)
    vac1sheet = writer.sheets['novali_es']    
    vac1sheet.write('D1', 'REGISTROS NO VALIDADOS', format1)
    ########
    df_snj.to_excel(writer, sheet_name='Vacuna_sin_jeringa',index=True, startrow=1, startcol=1)
    snjjsheet = writer.sheets['Vacuna_sin_jeringa']    
    snjjsheet.write('D1', 'REGISTROS DE VACUNAS SIN JERINGA', format1)
    ######## Con Diluyentye y Jeringa de dilusion####
    Con_dil.to_excel(writer, sheet_name='Vacuna_con_dil_jerdil',index=True, startrow=1, startcol=1)
    cndjdsheet = writer.sheets['Vacuna_con_dil_jerdil']    
    cndjdsheet.write('D1', 'REGISTROS CON DILUYENTE Y JERINGA DE DILUSION', format1)
     
    ###########
    for worksheet_name in writer.sheets:
        worksheet = workbook.get_worksheet_by_name(worksheet_name)
        worksheet.autofit()